In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *
#from fastai.transforms import *
from fastai.conv_learner import *
#from fastai.model import *
#from fastai.dataset import *
#from fastai.sgdr import *
#from fastai.plots import *

In [3]:
print(torch.__version__)

0.4.0


In [4]:
import pydicom
import glob
import os
import numpy
import seaborn as sns
from itertools import chain
from pathlib import Path

In [5]:
PATH = "./"
SPECTPATH = Path("../data/PPMISPECT/")

In [6]:
lstFilesDCM = []  # create an empty list
'''for dirName, subdirList, fileList in os.walk(PathDicom):
    for filename in fileList:
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM.append(os.path.join(dirName,filename))'''

'for dirName, subdirList, fileList in os.walk(PathDicom):\n    for filename in fileList:\n        if ".dcm" in filename.lower():  # check whether the file\'s DICOM\n            lstFilesDCM.append(os.path.join(dirName,filename))'

In [7]:
metadata = pd.read_csv(f'{PATH}PPMI.csv')

In [8]:
metadata[0:5]

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,388628,4140,Control,M,76,0,SPECT,Reconstructed DaTSCAN,Processed,4/09/2013,DCM,5/27/2018
1,388627,4139,Control,M,81,0,SPECT,Reconstructed DaTSCAN,Processed,3/19/2013,DCM,5/27/2018
2,388626,4137,PD,M,70,0,SPECT,Reconstructed DaTSCAN,Processed,4/02/2013,DCM,5/27/2018
3,388625,4136,PD,M,56,0,SPECT,Reconstructed DaTSCAN,Processed,2/13/2013,DCM,5/27/2018
4,451290,4135,PD,M,68,5,SPECT,Reconstructed DaTSCAN,Processed,2/18/2014,DCM,5/27/2018


In [9]:
#Shuffle data
metadata = metadata.sample(frac=1).reset_index(drop=True)

In [10]:
metadata[0:5]

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,360618,3769,Control,M,71,0,SPECT,Reconstructed DaTSCAN,Processed,10/25/2012,DCM,5/27/2018
1,388617,4122,PD,M,64,0,SPECT,Reconstructed DaTSCAN,Processed,2/06/2013,DCM,5/27/2018
2,363956,3870,PD,F,41,0,SPECT,Reconstructed DaTSCAN,Processed,11/29/2012,DCM,5/27/2018
3,388601,4107,PD,M,71,0,SPECT,Reconstructed DaTSCAN,Processed,12/21/2012,DCM,5/27/2018
4,419581,3829,PD,F,68,5,SPECT,Reconstructed DaTSCAN,Processed,10/01/2013,DCM,5/27/2018


In [11]:
for file in glob.glob(f'{PATH}PPMI*/**/Reconstructed_DaTSCAN/**/**/*.dcm'):
    lstFilesDCM.append(file)

In [12]:
len(lstFilesDCM)

1569

In [13]:
ptID = []
for i in lstFilesDCM:
    j = i.split("_")[13].strip()
    j = int(j[1:7])
    ptID.append(j)
len(ptID)

1569

In [14]:
p ={}
for o in ptID:
    for k in metadata.iloc[:,0]:
        if o == k:
            p[o] = metadata.loc[metadata['Image Data ID'] == k, 'Group'].tolist()[0]
len(p)

1569

In [15]:
df = metadata[['Image Data ID','Group','Subject','Age','Sex']].set_index('Image Data ID')
for index, rows in df.iterrows():
    for i in lstFilesDCM:
        j = i.split("_")[13].strip()
        j = int(j[1:7])
        if index == j:
            df.loc[df.index == j, "Filenames"] = i


In [163]:
#Check duplicate item
bg = []
for i in lstFilesDCM:
    for index, rows in df.iterrows():
        j = i.split("_")[13].strip()
        j = int(j[1:7])
        if j == index:
            bg.append(index)
print( [item for item, count in collections.Counter(bg).items() if count > 1] )
            

In [16]:
lstFilesDCM = df['Filenames'].values

In [17]:
lstFilesDCM = lstFilesDCM.tolist()

In [18]:
df.sample(frac=0.2, axis=0)

,Group,Subject,Age,Sex,Filenames
Image Data ID,,,,,
388578,Control,3969,80,F,./PPMI 9/3969/Reconstructed_DaTSCAN/2013-02-20...
767696,PD,3307,70,M,./PPMI 4/3307/Reconstructed_DaTSCAN/2015-11-05...
913228,PD,4109,71,M,./PPMI 4/4109/Reconstructed_DaTSCAN/2017-02-24...
418920,PD,3504,63,M,./PPMI 9/3504/Reconstructed_DaTSCAN/2013-01-23...
418669,PD,3307,67,M,./PPMI 2/3307/Reconstructed_DaTSCAN/2012-11-07...
360624,PD,4101,67,F,./PPMI 2/4101/Reconstructed_DaTSCAN/2012-11-14...
360615,PD,3633,69,F,./PPMI 3/3633/Reconstructed_DaTSCAN/2012-11-01...
342171,PD,3020,74,F,./PPMI 5/3020/Reconstructed_DaTSCAN/2012-03-27...
355210,PD,3473,55,F,./PPMI 3/3473/Reconstructed_DaTSCAN/2012-08-15...


In [19]:
up = pd.read_csv('comandfile.csv')

In [20]:
up.head()

,Image Data ID,Group,Filenames,TOTAL3,1YR,2YR,3YR,TOTAL3.1
0,688484,PD,./PPMI 5/3108/Reconstructed_DaTSCAN/2015-05-07...,10.500000,18.00,25.0,NaN,10.500000
1,446117,PD,./PPMI/3308/Reconstructed_DaTSCAN/2014-02-12_1...,19.666667,25.00,20.5,27.0,19.666667
2,446109,PD,./PPMI 9/3173/Reconstructed_DaTSCAN/2014-03-19...,24.500000,30.50,38.0,49.0,24.500000
3,341084,PD,./PPMI 8/3951/Reconstructed_DaTSCAN/2011-09-28...,14.000000,17.75,18.0,10.5,14.000000
4,363942,PD,./PPMI/3027/Reconstructed_DaTSCAN/2012-11-28_1...,26.000000,22.25,22.0,17.5,26.000000


In [21]:
up_df = up[['Image Data ID','TOTAL3','1YR','2YR','3YR','TOTAL3']].set_index('Image Data ID')

In [22]:
df1to4 = df.join(up_df)

In [23]:
df1to4 = df1to4.dropna(axis=0,subset=(['1YR']))

In [222]:
df1to4.to_csv('comandfile.csv')

In [7]:
df1to4 = pd.read_csv('comandfile.csv')

In [8]:
df1to4 = df1to4[~df1to4.Group.str.contains('Control')]

In [9]:
df1to4 = df1to4[~df1to4.TOTAL3.isnull()]

In [10]:
df1to4.any().isnull()

Unnamed: 0       False
Image Data ID    False
Group            False
Subject          False
Age              False
Sex              False
Filenames        False
TOTAL3           False
1YR              False
2YR              False
3YR              False
TOTAL3.1         False
dtype: bool

In [11]:
df1to4.loc[df1to4['Sex']=='F', 'Sex']=0
df1to4.loc[df1to4['Sex']=='M', 'Sex']=1

In [12]:
pro = (df1to4['1YR'].values - df1to4['TOTAL3'].values)
progress = []

for i in pro:
    if i > 2 : progress.append('1')
    elif i < (-2) : progress.append('0')
    else : progress.append('0')
df1to4['Progress'] = progress
df1to4['pro'] = pro
del(pro,progress)

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
scale3 = MinMaxScaler()

In [15]:
print(scale3.fit(df1to4['TOTAL3'].values.reshape(-1,1)))

MinMaxScaler(copy=True, feature_range=(0, 1))


In [16]:
df1to4['3scale'] = scale3.transform(df1to4['TOTAL3'].values.reshape(-1,1))

In [17]:
del(scale3)

In [18]:
from sklearn.model_selection import train_test_split
X_train_tmp, X_test_tmp, param_train_tmp, param_test_tmp, y_train_tmp, y_test_tmp = train_test_split(df1to4, df1to4[['Sex','Age','3scale']], df1to4['Progress'], test_size=0.2)

from sklearn.model_selection import train_test_split
X_train_tmp, X_val_tmp, param_train_tmp, param_val_tmp, y_train_tmp, y_val_tmp = train_test_split(X_train_tmp, param_train_tmp, y_train_tmp, test_size=0.3)

In [19]:
lstFilesDCM_train = []  # create an empty list
lstFilesDCM_test = []
lstFilesDCM_val = []
lstFilesDCM_train = X_train_tmp['Filenames'].values
lstFilesDCM_train = lstFilesDCM_train.tolist()
lstFilesDCM_test = X_test_tmp['Filenames'].values
lstFilesDCM_test = lstFilesDCM_test.tolist()
lstFilesDCM_val = X_val_tmp['Filenames'].values
lstFilesDCM_val = lstFilesDCM_val.tolist()

For NORMAL Image

In [20]:
#Training Set

In [21]:
# loop through all the DICOM files
image_array_train =[]

for filenameDCM in lstFilesDCM_train:
    ds = pydicom.read_file(str(SPECTPATH / filenameDCM))
    ds_3d = ds.pixel_array[30]
    for i in range(31,51):
        ds_3d = np.dstack((ds_3d,ds.pixel_array[i]))

    image_array_train.append(ds_3d)

image_array_train = np.asarray(image_array_train)
image_array_train = image_array_train.transpose(0,3,1,2)
image_array_train.shape

(685, 21, 109, 91)

In [22]:
image_array_train= image_array_train.astype('float64')

In [23]:
image_array_train -= image_array_train.min()
image_array_train /= (image_array_train.max()-image_array_train.min())

In [24]:
image_array_train.shape

(685, 21, 109, 91)

In [25]:
#Validation set

In [26]:
# loop through all the DICOM files
image_array_val =[]

for filenameDCM in lstFilesDCM_val:
    ds = pydicom.read_file(str(SPECTPATH / filenameDCM))
    ds_3d = ds.pixel_array[30]
    for i in range(31,51):
        ds_3d = np.dstack((ds_3d,ds.pixel_array[i]))

    image_array_val.append(ds_3d)

image_array_val = np.asarray(image_array_val, dtype='float64')
image_array_val = image_array_val.transpose(0,3,1,2)
image_array_val -= image_array_val.min()
image_array_val /= (image_array_val.max()-image_array_val.min())
image_array_val.shape

(294, 21, 109, 91)

In [27]:
#Test set

In [28]:
# loop through all the DICOM files
image_array_test =[]

for filenameDCM in lstFilesDCM_test:
    ds = pydicom.read_file(str(SPECTPATH / filenameDCM))
    ds_3d = ds.pixel_array[30]
    for i in range(31,51):
        ds_3d = np.dstack((ds_3d,ds.pixel_array[i]))

    image_array_test.append(ds_3d)

image_array_test = np.asarray(image_array_test, dtype='float64')
image_array_val = image_array_val.transpose(0,3,1,2)
image_array_test -= image_array_test.min()
image_array_test /= (image_array_test.max()-image_array_test.min())
image_array_test.shape 

(245, 21, 109, 91)

In [29]:
#Check matched data-result by Image ID number
lstFilesDCM_test[0:5]

['./PPMI5/3023/Reconstructed_DaTSCAN/2014-08-28_14_40_25.0/S237562/PPMI_3023_NM_Reconstructed_DaTSCAN_Br_20150112152112798_1_S237562_I468262.dcm',
 './PPMI4/3118/Reconstructed_DaTSCAN/2014-02-20_14_02_11.0/S231150/PPMI_3118_NM_Reconstructed_DaTSCAN_Br_20141006104513369_1_S231150_I446107.dcm',
 './PPMI3.5/3054/Reconstructed_DaTSCAN/2014-12-17_14_37_14.0/S561082/PPMI_3054_NM_Reconstructed_DaTSCAN_Br_20170503135055345_1_S561082_I846852.dcm',
 './PPMI/3588/Reconstructed_DaTSCAN/2014-10-07_12_10_13.0/S244219/PPMI_3588_NM_Reconstructed_DaTSCAN_Br_20150807124645849_1_S244219_I504433.dcm',
 './PPMI7/3621/Reconstructed_DaTSCAN/2013-03-01_14_20_10.0/S202934/PPMI_3621_NM_Reconstructed_DaTSCAN_Br_20140403144458594_1_S202934_I418693.dcm']

In [30]:
X_test_tmp[0:5]

,Unnamed: 0,Image Data ID,Group,Subject,Age,Sex,Filenames,TOTAL3,1YR,2YR,3YR,TOTAL3.1,Progress,pro,3scale
1103,1103,468262,PD,3023,73,0,./PPMI5/3023/Reconstructed_DaTSCAN/2014-08-28_...,33.000000,34.0,38.0,NaN,33.000000,0,1.000000,0.410596
198,198,446107,PD,3118,62,1,./PPMI4/3118/Reconstructed_DaTSCAN/2014-02-20_...,22.000000,21.5,12.5,44.0,22.000000,0,-0.500000,0.264901
330,330,846852,PD,3054,78,1,./PPMI3.5/3054/Reconstructed_DaTSCAN/2014-12-1...,26.666667,31.0,51.0,44.0,26.666667,1,4.333333,0.326711
833,833,504433,PD,3588,51,0,./PPMI/3588/Reconstructed_DaTSCAN/2014-10-07_1...,39.500000,46.5,46.0,42.0,39.500000,1,7.000000,0.496689
875,875,418693,PD,3621,55,0,./PPMI7/3621/Reconstructed_DaTSCAN/2013-03-01_...,16.666667,14.0,17.0,NaN,16.666667,0,-2.666667,0.194260


In [29]:
y_train = np.asarray(y_train_tmp.values)
y_train = y_train.astype('int64')
y_val = np.asarray(y_val_tmp.values)
y_val = y_val.astype('int64')
y_test = np.asarray(y_test_tmp.values)
y_test = y_test.astype('int64')

In [30]:
y_train.shape,y_val.shape,y_test.shape

((685,), (294,), (245,))

In [31]:
param_train = param_train_tmp.values
params = []
for i in param_train:
    stack = []
    stack.append(i)
    stack = stack*10
    params.append(stack)
param_train = np.asarray(params)
del(params)

In [32]:
param_val = param_val_tmp.values
params = []
for i in param_val:
    stack = []
    stack.append(i)
    stack = stack*10
    params.append(stack)
param_val = np.asarray(params)
del(params)

param_test = param_test_tmp.values
params = []
for i in param_test:
    stack = []
    stack.append(i)
    stack = stack*10
    params.append(stack)
param_test = np.asarray(params)
del(params)

In [33]:
param_train.shape

(685, 10, 3)

### Load dataset

In [36]:
param_train = torch.from_numpy(param_train)
param_test = torch.from_numpy(param_test)
param_val = torch.from_numpy(param_val)
image_array_train = torch.from_numpy(image_array_train)
image_array_test = torch.from_numpy(image_array_test)
image_array_val = torch.from_numpy(image_array_val)
y_train = torch.from_numpy(y_train)
y_val = torch.from_numpy(y_val)
y_test = torch.from_numpy(y_test)

In [34]:
X_train = torch.utils.data.DataLoader(image_array_train, batch_size=4)
param_train = torch.utils.data.DataLoader(param_train, batch_size=4)
y_train = torch.utils.data.DataLoader(y_train, batch_size=4)
X_val = torch.utils.data.DataLoader(image_array_val, batch_size=4)
param_val = torch.utils.data.DataLoader(param_val, batch_size=4)
y_val = torch.utils.data.DataLoader(y_val, batch_size=4)
X_test = torch.utils.data.DataLoader(image_array_test, batch_size=4)
param_test = torch.utils.data.DataLoader(param_test, batch_size=4)
y_test = torch.utils.data.DataLoader(y_test, batch_size=4)

del(image_array_train,image_array_val,image_array_test)
del(X_train_tmp,X_val_tmp,X_test_tmp,y_train_tmp,y_val_tmp,y_test_tmp,param_train_tmp,param_val_tmp,param_test_tmp)

### Darknet

In [46]:
def conv_layer(ni, nf, ks=3, stride=1):
    return nn.Sequential(
        nn.Conv3d(ni, nf, kernel_size=ks, bias=False, stride=stride, padding=ks//2),
        nn.BatchNorm3d(nf, momentum=0.01),
        nn.LeakyReLU(negative_slope=0.1, inplace=False))

class ResLayer(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.conv1=conv_layer(ni, ni//2, ks=1)
        self.conv2=conv_layer(ni//2, ni, ks=3)
        
    def forward(self, x): return x.add_(self.conv2(self.conv1(x.clone())))

class Darknet(nn.Module):
    def make_group_layer(self, ch_in, num_blocks, stride=1):
        return [conv_layer(ch_in, ch_in*2,stride=stride)
               ] + [(ResLayer(ch_in*2)) for i in range(num_blocks)]

    def __init__(self, num_blocks, num_classes, nf=32):
        super().__init__()
        features = [conv_layer(1, nf, ks=3, stride=1)]
        for i,nb in enumerate(num_blocks):
            features += self.make_group_layer(nf, nb, stride=2-(i==1))
            nf *= 2                        
        features += [nn.AdaptiveAvgPool3d((2,4,4)), Flatten()]        
        self.features = nn.Sequential(*features)
        fc1 = [nn.Linear(16414, 320), nn.LeakyReLU(negative_slope=0.1, inplace=False)]
        self.fc1 = nn.Sequential(*fc1)
        self.layers = nn.Sequential(nn.Linear(320,1))
        
    def forward(self, x, y):
        x = x.unsqueeze(1)
        x = self.features(x)
        x = torch.cat((x,y[:,:,0],y[:,:,1],y[:,:,2]),1)
        x = self.fc1(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.2, training=self.training)
        #return F.log_softmax(self.layers(x), dim=-1)
        return self.layers(x)


In [47]:
m = Darknet([1,2,4,3], num_classes=1, nf=32)
m.double()
#m = nn.DataParallel(m, [1,2,3])

Darknet(
  (features): Sequential(
    (0): Sequential(
      (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (1): BatchNorm3d(32, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
    )
    (1): Sequential(
      (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
    )
    (2): ResLayer(
      (conv1): Sequential(
        (0): Conv3d(64, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (1): BatchNorm3d(32, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.1)
      )
      (conv2): Sequential(
        (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.

In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
m.to(device)
print(device)

cuda:0


In [58]:
optimizer = optim.Adam(m.parameters(), lr = 1e-4)
criterion = nn.BCEWithLogitsLoss().cuda()
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.5)

In [57]:
losslog=[]
vallog=[]

In [51]:
torch.backends.cudnn.benchmark=True

In [52]:
def train(niter):
    for epoch in trange(niter):  # loop over the dataset multiple times
        running_loss = 0.0        

        m.train(True)
        scheduler.step()
        with tqdm(total=len(X_train)) as pbar:
            for i, (x, p, y) in enumerate(zip(X_train,param_train,y_train)):
                x,p,y = x.to(device), p.to(device),y.to(device)
                #i += 1
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = m(x,p)
                loss = criterion(outputs,y.double().unsqueeze(1))
                
                loss.backward()
                for param in optimizer.param_groups[0]['params']:
                    param.data = param.data.add(-0.0001 * optimizer.param_groups[0]['lr'], param.data)
                optimizer.step()
                pbar.update()

                # print statistics
                running_loss += loss.item()
                if i % 20 == 19:    # print every 0 mini-batches
                    print('[%d, %5d]  Train BCELogitsLoss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 20))
                    losslog.append(running_loss/20)
                    running_loss = 0.0
                    
            #Eval
        m.eval()
        with tqdm(total=len(X_val)) as pbar:
            with torch.no_grad():
                truth = []
                predicted =[]
                correct = 0
                total = 0
                for i, ( x, p, y) in enumerate(zip(X_val,param_val,y_val)):
                    x,p,y = x.to(device), p.to(device),y.to(device)

                    # forward + backward + optimize
                    outputs = m(x,p)
                    pred = (F.sigmoid(outputs))
                    corr=[]
                    for i in pred:
                        if i >= 0.68: corr.append(1)
                        else: corr.append(0)
                    total += y.size(0)
                    correct += (corr == y.cpu().numpy()).sum()
                    predicted.append(pred)
                    truth.append(y)
                    pbar.update()

                valloss = loss.item()
                vallog.append(valloss)
                print(f'BCELogitsLoss: {valloss}')
                print('Accuracy of the network on the val images: %d %%' % (100 * correct / total))
            
    print('Finished Training')
    return losslog,vallog,truth,predicted    loss = criterion(outputs,y.double().unsqueeze(1))
                

In [59]:
MUL = train(2)

100%|██████████| 74/74 [00:50<00:00,  1.71it/s]BCELogitsLoss: 0.7280747258520811
Accuracy of the network on the val images: 54 %

100%|██████████| 74/74 [00:50<00:00,  1.70it/s]BCELogitsLoss: 0.7119796252634321
Accuracy of the network on the val images: 54 %

100%|██████████| 2/2 [15:12<00:00, 456.01s/it]Finished Training


In [54]:
vallog

[0.8094666410848532, 0.7621253101425307]

In [60]:
MUL

([0.6899742491367287,
  0.7216632823748271,
  0.7031199616145714,
  0.6551904824257477,
  0.7400505342567856,
  0.7045912965095784,
  0.6957142097621987,
  0.716587152393308,
  0.6838836016871113,
  0.7017663175325224,
  0.6961114322213378,
  0.664758787050066,
  0.7317512962402098,
  0.6759753116480365,
  0.6937985015888914,
  0.7016916941138533],
 [0.7280747258520811, 0.7119796252634321])

In [71]:
def eva():    
    m.eval()
    with tqdm(total=len(X_val)) as pbar:
        with torch.no_grad():
            truth = []
            predicted =[]
            correct = 0
            total = 0
            for i, ( x, p, y) in enumerate(zip(X_val,param_val,y_val)):
                x,p,y = x.to(device), p.to(device),y.to(device)

                # forward + backward + optimize
                outputs = m(x,p)
                pred = (F.sigmoid(outputs))
                corr=[]
                for i in pred:
                    if i >= 0.52: corr.append(1)
                    else: corr.append(0)
                total += y.size(0)
                correct += (corr == y.cpu().numpy()).sum()
                predicted.append(pred)
                truth.append(y)
                pbar.update()


            print('Accuracy of the network on the val images: %d %%' % (100 * correct / total))
            
    print('Finished Training')
    return losslog,vallog,truth,predicted    
            

In [72]:
M = eva()

100%|██████████| 74/74 [00:50<00:00,  1.72it/s]Accuracy of the network on the val images: 56 %

Finished Training


In [74]:
for i in range(20,50,2):
    print(i)

20
22
24
26
28
30
32
34
36
38
40
42
44
46
48


In [ ]:


## Validate

In [52]:
def test():
    with torch.no_grad():
        n = len(X_test)
        ml.eval()
        predicted = []
        correct = 0
        total = 0
        with tqdm(total=n) as pbar:
            for i, ( x, p, y) in enumerate(zip(X_test,param_test,y_test)):
                x,p,y = x.to(device), p.to(device),y.to(device)

                # forward + backward + optimize
                outputs = ml(x,p)
                _, pred = torch.max(outputs.data, 1)
                total += y.size(0)
                correct += (pred == y).sum()
    
    print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))
    return pred

In [ ]:
TEST = test()

  0%|          | 0/123 [00:00<?, ?it/s]

In [52]:
TEST

tensor([ 1], device='cuda:0')

In [50]:
model_save_path = './weights/'

In [51]:
torch.save(ml.state_dict(),f'{model_save_path}Predict_progress_3D.pth')

In [46]:
ml = Darknet([1,2,4,6,3], num_classes=1, nf=32)
ml.double()
ml.load_state_dict(torch.load('Predict_pregress_.pth'))

In [47]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ml.to(device)
print(device)

cuda:0
